# HBase基本操作

### 一、实验介绍

#### 1.1 实验内容

- HBase的概述及历史
- HBase的数据模型
- HBase的系统架构
- HBase安装模式介绍与配置
- HBase的启动/停止
- HBase shell
- DDL与DML操作

#### 1.2 实验知识点

- 了解hbase概念
- 把握Hbase数据模型，编程接口
- 知道hbase的常见应用场景
- hbase shell的使用
- 掌握Hbase的安装模式
- hadoop启动及相关进程
- hbase shell的常用命令
- Hbase的启动
- hadoop表的操作

#### 1.3 实验环境

- jupyter terminal终端
- hadoop2.6.0
- hbase0.98

#### 1.4 适合人群

本课程难度为一般，属于初级级别课程，适合具有hadoop基础的用户。

### 二、HBase概述

HBase是一个开源的非关系型分布式数据库（NoSQL），它参考了谷歌的BigTable建模，实现的编程语言为 Java。它是Apache软件基金会的Hadoop项目的一部分，运行于HDFS文件系统之上，为 Hadoop 提供类似于BigTable 规模的服务，可以存储海量稀疏的数据，并具备一定的容错性、高可靠性及伸缩性。主要应用场景是实时随机读写超大规模的数据。

HBase在列上实现了BigTable论文提到的压缩算法、内存操作和布隆过滤器。HBase的表能够作为MapReduce任务的输入和输出，可以通过Java API来存取数据，也可以通过REST、Avro或者Thrift的API来访问。

HBase不能取代RDBMS，因为二者的应用场景不同。HBase为了解决海量数据的扩展性，支持简单的增加节点来实现线性扩展，从而在集群上管理海量的非结构化或半结构化的稀疏数据。HBase仅能通过主键（raw key）或主键的range检索数据，支持单行事务。
![](http://onm3nv9z1.bkt.clouddn.com/Hadoop%20EcoSystem.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
上图描述Hadoop EcoSystem中的各层系统。其中,HBase位于结构化存储层，Hadoop HDFS为HBase提供了高可靠性的底层存储支持，Hadoop MapReduce为HBase提供了高性能的计算能力，Zookeeper为HBase提供了稳定服务和failover机制。

此外，Pig和Hive还为HBase提供了高层语言支持，使得在HBase上进行数据统计处理变的非常简单。 Sqoop则为HBase提供了方便的RDBMS数据导入功能，使得传统数据库数据向HBase中迁移变的非常方便。

### 三、HBase历史

Apache HBase最初是Powerset公司为了处理自然语言搜索产生的海量数据而开展的项目。下图展示了HBase的发展历程。
![](http://onm3nv9z1.bkt.clouddn.com/Hbase-history.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

### 四、HBase数据模型

![](http://onm3nv9z1.bkt.clouddn.com/Hbase-data-model.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
- 行健（Row Key）：表的主键，表中的记录默认按照行健升序排序
- 时间戳（Timestamp）：每次数据操作对应的时间戳，可以看作是数据的版本号
- 列族（Column Family）：表在水平方向有一个或者多个列族组成，一个列族中可以由任意多个列组成，列族支持动态扩展，无需预先定义列的数量以及类型，所有列均以二进制格式存储，用户需要自行进行类型转换。所有的列族成员的前缀是相同的，例如“abc:a1”和“abc:a2”两个列都属于abc这个列族。
- 表和区域（Table&Region）：当表随着记录数不断增加而变大后，会逐渐分裂成多份，成为区域，一个区域是对表的水平划分，不同的区域会被Master分配给相应的RegionServer进行管理
- 单元格（Cell）：表存储数据的单元。由{行健，列（列族:标签），时间戳}唯一确定，其中的数据是没有类型的，以二进制的形式存储。

### 五、HBase架构

![](http://onm3nv9z1.bkt.clouddn.com/hbase-framework.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
如上图所示，HBase架构中只有一个Master节点，称HMaster，还有多台RegionServer成为HRegionServer，每个RegionServer包含多个Region。

- Native Java API，最常规和高效的访问方式，适合Hadoop MapReduce Job并行批处理HBase表数据
- HBase Shell，HBase的命令行工具，最简单的接口，适合HBase管理使用
- Thrift Gateway，利用Thrift序列化技术，支持C++，PHP，Python等多种语言，适合其他异构系统在线访问HBase表数据
- REST Gateway，支持REST 风格的Http API访问HBase, 解除了语言限制
- Pig，可以使用Pig Latin流式编程语言来操作HBase中的数据，和Hive类似，本质最终也是编译成MapReduce Job来处理HBase表数据，适合做数据统计
- Hive，当前Hive的Release版本尚没有加入对HBase的支持，但在下一个版本Hive 0.7.0中将会支持HBase，可以使用类似SQL语言来访问HBase

### 六、HBase存储格式

HBase中的所有数据文件都存储在Hadoop HDFS文件系统上，主要包括上述提出的两种文件类型：

- HFile， HBase中KeyValue数据的存储格式，HFile是Hadoop的二进制格式文件，实际上StoreFile就是对HFile做了轻量级包装，即StoreFile底层就是HFile
- HLogFile，HBase中WAL（Write Ahead Log） 的存储格式，物理上是Hadoop的Sequence File

### 七、HBase应用场景

#### HBase的优势主要在以下几方面：
- 海量数据存储
- 快速随机访问
- 大量写操作的应用

#### 常见的应用场景
- 互联网搜索引擎数据存储（BigTable要解决的问题）
- 审计日志系统
- 实时系统
- 消息中心
- 内容服务系统
### 八、参考资料

- http://www.ymc.ch/en/introduction-to-hbase
- http://bigdatariding.blogspot.jp/2013/12/hbase-architecture.html
- http://baike.baidu.com/view/1993870.htm
- http://hbase.apache.org/
- http://www.alidata.org/archives/1509
- http://zh.wikipedia.org/zh/Apache_HBase
- http://abloz.com/hbase/book.html





## 基本操作
#### 一、Hbase Shell
本章介绍HBase Shell 的一些常用命令。
#### 1.1 实验准备
- 下载安装 Hbase（该步骤已由稀牛完成）

上节课程我们已经了解到，HBase是一个开源的非关系型分布式数据库（NoSQL），它是Apache软件基金会的Hadoop项目的一部分，运行于HDFS文件系统之上。因此，安装Hbase之前必须确保 Hadoop 已经成功安装。本次实验，使用HBase V0.98.11版本。

- 启动HBase，进入实验环境

①首先开启ssh

sudo service ssh start

②启动Hadoop

sudo su - hadoop

start-all.sh

用jps命令查看进程，查看Hadoop启动是否成功

![](http://onm3nv9z1.bkt.clouddn.com/hadoop-jps.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
③启动HBase

cd hbase

bin/start-hbase.sh
![](http://onm3nv9z1.bkt.clouddn.com/hbase-start.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
使用jps命令查看进程
![](http://onm3nv9z1.bkt.clouddn.com/hbase-jps.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
#### 1.2 连接HBase
使用hbase shell命令来连接正在运行的Hbase实例，该命令位于HBase安装包下的bin/目录。HBase Shell提示符以>符号结束。
![](http://onm3nv9z1.bkt.clouddn.com/hbase_shell_login.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 1.3 显示HBase Shell 帮助文档
输入help并按Enter键，可以显示HBase Shell的基本使用信息，和我们接下来会列举的一些命令类似。需要注意的是，表名，行，列都必须包含在引号内。

#### 1.4 退出HBase Shell
使用quit命令，退出HBase Shell 并且断开和集群的连接，但此时HBase仍然在后台运行。

#### 1.5 查看HBase状态

![](http://onm3nv9z1.bkt.clouddn.com/hbase_shell_status.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 1.6查看当前用户

使用命令whoami
![](http://onm3nv9z1.bkt.clouddn.com/hbase-whoami.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

####  1.7查看HBase版本

使用命令version
![](http://onm3nv9z1.bkt.clouddn.com/hbase-version.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 1.8关闭HBase
和bin/start-hbase.sh开启所有的HBase进程相同，bin/stop-hbase.sh用于关闭所有的HBase进程。

### 二、数据定义（DDL）操作

##### 2.1创建新表

使用create命令来创建一个新的表。在创建的时候，必须指定表名和列族名。

这里我们使用的表名为student，列族只有一个info。
![](http://onm3nv9z1.bkt.clouddn.com/hbase-create-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

##### 2.2 列举表信息
使用list命令：
![](http://onm3nv9z1.bkt.clouddn.com/hbase-list-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
#### 2.3 获取表描述
使用describe命令：
![](http://onm3nv9z1.bkt.clouddn.com/hbase-describe-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
#### 2.4 删除表
删除表之前，先disable表,再使用drop命令实现删除表的功能：
![](http://onm3nv9z1.bkt.clouddn.com/hbase-drop-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
#### 2.5 检查表是否存在
![](http://onm3nv9z1.bkt.clouddn.com/hbase-exists-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

### 三、数据管理（DML）操作
#### 3.1 向表中插入数据
使用put命令，将数据插入表中：

put 'student','Allen','info:age','25'

put 'student','Allen','info:sex','m'

put 'student','Allen','info:city','New York'

put 'student','Bob','info:age','23'

put 'student','Bob','info:sex','m'

put 'student','Bob','info:city','Chicago'

put 'student','Cristina','info:age','23'

put 'student','Cristina','info:sex','f'

put 'student','Cristina','info:city','New York'

可以看到，在本例中，一共插入了九条数据，一次一条。第一次插入到student行，info:age列，插入值为25。所有列在HBase中有一个列族前缀。本例中的info，后面跟着一个冒号还有一个列限定后缀，本例中是age。

#### 3.2 获取一个行数据
使用get命令来获得某一行的数据：

get 'student','Allen' 
![](http://onm3nv9z1.bkt.clouddn.com/hbase-get-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

可以加上列族、列族+列限定符，来获取更精确的行数据：

get 'student','Allen','info'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-get-info-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

get 'student','Allen','info:city'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-get-info-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 3.3 修改行数据
使用put命令修改行数据。

put 'student','Bob','info:age','24'

get 'student','Bob','info:age'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-put-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
#### 3.4 一次性扫描全表数据

一种获取HBase数据的方法是扫描，使用scan命令来扫描表的数据。可以限制限制扫描的范围，在本例中，获取的是所有的数据。
![](http://onm3nv9z1.bkt.clouddn.com/hbase-scan-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 3.5 对表计数

count 'student'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-count-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 3.6 删除列族

delete 'student','Cristina','info:sex'

get 'student','Cristina'

![](http://onm3nv9z1.bkt.clouddn.com/hbase-delete-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 3.7 删除行

deleteall 'student','Cristina'

get 'student','Cristina'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-deleteall-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

#### 3.8 删除表

首先用命令查看是否存在表student

exists 'student'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-exists-student1.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
尝试删除表，提示删除失败

drop 'student'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-drop-student1.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)

因为在删除之前需要先禁用表。

is_enable 'student'

disable 'student'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-is_enable-student.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
再次尝试删除表，提示成功

drop 'student'

exists 'student'
![](http://onm3nv9z1.bkt.clouddn.com/hbase-drop-student2.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
### 十一、实验总结
本章介绍了HBase Shell的一些基本的常用命令，主要包括数据定义（DLL）的数据管理（DML）的一些操作命令。只能满足基本的HBase操作，更进一步的了解，请通过阅读HBase官方教程。


### 四、参考资料

- http://blog.csdn.net/gnail_oug/article/details/46997997
- http://www.yiibai.com/hbase/hbase_installation.html#article-start
- http://hbase.apache.org/book.html


### 十三、练习题

1.HBase属于列数据库，与普通行数据库的区别是什么？

2.数据如下图形式，请设计并建表，插入以下数据。给出对表scan命令的执行结果提交。
![](http://onm3nv9z1.bkt.clouddn.com/hbase-test1.png?imageView2/0/q/75|watermark/2/text/56iA54mb5a6e6aqM5a6k/font/5a6L5L2T/fontsize/500/fill/IzRCRjYwOA==/dissolve/100/gravity/SouthEast/dx/10/dy/10|imageslim)
---
请将你的练习答案写在这里：



In [ ]:
1.
a. HBase的每一行由唯一的行键确定

在某种程度上，行键相当于传统数据库的primary key，区别在于，primary key是可选的，而HBase的每张表都必然会有行键。除了行键之外，HBase表不能对列添加索引。HBase是一个<key, value>形式的数据库，行键就是它的key。

b. HBase引入了列族(columnFamily)的概念

HBase是一个列式存储的数据库，因此列的使用是非常灵活的，不必在表定义的时候就定好列名，但是必须在建表的时候定义好列族名字。
一张HBase表存储的列的数量可以是无限的，但是列族的数量最好控制在2-3个（原因在备注中[1]）
列必须属于某个列族，不同列族之间可以有同名列
列族的作用是，将那些数据量和属性相似的列聚集在一起，以便我们给这些列定义一些共同的存储方式属性(e.g. 数据压缩，保存到读缓存中)
c. HBase的列值可以有多版本

在HBase表中，行键、列族、列名和时间戳才能唯一确定一个值。每个值是一个单元格(cell)，是存储的基本单位。每一行数据的每一列，都可以存储多个值，每个版本的值之间通过时间戳确定，在存储的时候，这些值也按照时间戳逆序排列，保证客户端永远读到最新的数据。但是，每个列族可以存储的最大版本数是确定的，并且是在建表的时候就定义好的。

另外，单元格的时间戳是可以由用户自行指定的，如果不指定，服务器就会将接收到写请求的服务器时间作为单元格的时间戳。通常情况下，最好不要自己指定时间戳，因为客户端总是难以保证，指定的时间戳是按照写顺序递增的。

d. 反范式化

HBase是一个NoSQL(Not-only-SQL)数据库，不提供复杂的查询方式，包括join。另外，相对于MySQL，HBase的可扩展性很好，存储资源要廉价很多。因此，在设计数据库的时候，我们总是倾向于反范式化，以方便后期的数据查询

2.
ROW                               COLUMN+CELL
 2017007                          column=info:age, timestamp=1509894209259, value=25
 2017007                          column=info:city, timestamp=1509894239370, value=San francisco
 2017007                          column=info:sex, timestamp=1509894218283, value=m
 2017007                          column=name:, timestamp=1509894252480, value=Donald
 2017008                          column=info:age, timestamp=1509894276744, value=26
 2017008                          column=info:city, timestamp=1509894301665, value=New York
 2017008                          column=info:sex, timestamp=1509894287546, value=m
 2017008                          column=name:, timestamp=1509894260041, value=Donald
 2017009                          column=info:age, timestamp=1509894334125, value=24
 2017009                          column=info:city, timestamp=1509894356536, value=Los Angeles
 2017009                          column=info:sex, timestamp=1509894341410, value=m
 2017009                          column=name:, timestamp=1509894319520, value=Edison
3 row(s) in 0.0360 seconds